In [67]:
import numpy as np
import numpy.random as random

In [7]:
#Initialize parameters
K = 10 #number of topics
V = 1000 #number of words
A = 2 #dimension of topical content
N = 20 #number of documents

In [131]:
mu = np.array([0]*(K-1))
sigma = np.zeros(((K-1),(K-1)))
diag = np.diagonal(sigma, 0)
diag.setflags(write=True)
diag.fill(20)
beta = random.gamma(.1,1, V*K).reshape(K,V)
beta = (beta.T/beta.sum(axis=1)).T
lambd = np.zeros((N, (K-1)))
beta = np.repeat(list(beta),A)

In [152]:
model = {'mu':mu, 'sigma':sigma, 'beta': beta, 'lambda': lambd}